In [ ]:
# Installing LLAMA, Groq
!pip install -q llama-index==0.10.14
!pip install llama-index-llms-groq
from llama_index.llms.groq import Groq
!pip install -q gradio
import gradio as gr
import markdown
import time
!pip install groq
!pip install socketify pyngrok
!pip install nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py)

In [ ]:
# Install necessary packages
!pip install fastapi uvicorn pyngrok

In [ ]:
# First, let's install the necessary package: rake-nltk - similarity document
!pip install rake-nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Import Depedencies
import uvicorn
from pyngrok import ngrok
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rake_nltk import Rake
from nltk.corpus import stopwords
import nest_asyncio
from fastapi import FastAPI, HTTPException, Form
from pydantic import BaseModel
from llama_index.core.llms import ChatMessage
from llama_index.llms.groq import Groq
import re
from typing import List
import random
nest_asyncio.apply()

app = FastAPI()

# Similarity Texts
# RAKE - TF-IDF - Cosine Similarity
# Define a Pydantic model for the input data
class Story(BaseModel):
    id_story: str
    orientation: str
    complication: str
    resolution: str
    reorientation: str
    kode_kelompok: str

class Stories(BaseModel):
    lists: List[Story]

# Function to calculate similarity using RAKE and TF-IDF
def texts_similarity_with_rake(stories):
    # Initialize RAKE using NLTK's stopwords
    rake = Rake(stopwords=stopwords.words('english'))
    prepared_texts = []

    # Extract and prepare key phrases for all texts
    for story in stories:
        combined_text = f"{story.orientation} {story.complication} {story.resolution} {story.reorientation}"
        rake.extract_keywords_from_text(combined_text)
        key_phrases = rake.get_ranked_phrases()
        prepared_text = " ".join(key_phrases)
        prepared_texts.append(prepared_text)

    # Create TF-IDF vectors for all prepared texts
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(prepared_texts)

    # Calculate the cosine similarity matrix
    similarity_matrix = cosine_similarity(vectors)

    return similarity_matrix

# Define the API endpoint
@app.post("/v1/grading-similarity-document")
async def calculate_similarity(stories: Stories):
    similarity_matrix = texts_similarity_with_rake(stories.lists)
    results = []

    for i in range(len(stories.lists)):
        for j in range(i + 1, len(stories.lists)):
            result = {
                "id_story_1": stories.lists[i].id_story,
                "orientation_1": stories.lists[i].orientation,
                "complication_1": stories.lists[i].complication,
                "resolution_1": stories.lists[i].resolution,
                "reorientation_1": stories.lists[i].reorientation,
                "kode_kelompok_1": stories.lists[i].kode_kelompok,
                "id_story_2": stories.lists[j].id_story,
                "orientation_2": stories.lists[j].orientation,
                "complication_2": stories.lists[j].complication,
                "resolution_2": stories.lists[j].resolution,
                "reorientation_2": stories.lists[j].reorientation,
                "kode_kelompok_2": stories.lists[j].kode_kelompok,
                "similarity_score": similarity_matrix[i][j]
            }
            results.append(result)

    return {"similarity_results": results}

# QUIZ
class Quiz(BaseModel):
    document: str
    question: str
    choices: list
    correct_field: str

class Answer(BaseModel):
    quiz: list[Quiz]
    user_answers: list[int]

class DocumentData(BaseModel):
    data: dict

# Create a quiz from the JSON data
def create_quiz(data, num_quizzes=5):
    fields = ["Orientasi", "Complication", "Resolution", "Reorientation", "Recomplication"]
    documents = list(data.keys())
    quizzes = []

    for _ in range(num_quizzes):
        selected_doc = random.choice(documents)
        selected_field = random.choice(fields)
        correct_answer = data[selected_doc][selected_field]
        quiz = {
            'document': selected_doc,
            'question': correct_answer,
            'choices': fields,
            'correct_field': selected_field
        }
        quizzes.append(quiz)

    return quizzes

# Evaluate the quiz
def evaluate_quiz(quiz, user_answers):
    score = 0
    total = len(quiz)

    evaluation_result = []
    for q, user_choice_index in zip(quiz, user_answers):
        user_answer = q.choices[user_choice_index]
        correct_answer = q.correct_field
        is_correct = user_answer == correct_answer
        if is_correct:
            score += 1
        evaluation_result.append({
            'question': q.question,
            'user_answer': user_answer,
            'correct_answer': correct_answer,
            'is_correct': is_correct
        })

    return score, total, evaluation_result

# MAKE QUIZ
@app.post("/v1/make-quiz")
def make_quiz(document_data: DocumentData, num_quizzes: int = 5):
    data = document_data.data
    quizzes = create_quiz(data, num_quizzes)
    return {"quizzes": quizzes}

# CHECK ANSWERS
@app.post("/v1/check-answer")
def check_answer(answer: Answer):
    score, total, evaluation_result = evaluate_quiz(answer.quiz, answer.user_answers)
    text_total_score = f"{score}/{total}"
    grade_score = score / total * 100
    return {
        "total_score": text_total_score,
        "grade_score": grade_score,
        "evaluation_result": evaluation_result
    }

# LLM Groq Update
# Define the request model for input text
class LLMRequest(BaseModel):
    input_text: str

# LLM RECOMENDATION SCORE

# Api of grading
@app.post("/v1/grading-recommendation")
async def process_text(request: LLMRequest):
    llm = Groq(model="mixtral-8x7b-32768", api_key="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")

    prompt = """
    As a specialized assistant in teaching and evaluating narrative text structure, your task entails assessing the provided narrative based on Orientation, Complication, Resolution, and Reorientation,
    devoid of extensive preamble. Each segment will be scrutinized for quality, with specific examples provided, and at least five detailed suggestions for improvement offered. Grammatical or syntactical
    errors will be identified, along with clear recommendations for correction. The significance of each mistake will be highlighted, elucidating how rectifying them could enhance the overall text quality.
    The evaluation will culminate in assigning a final grade, within even numbers ranging between 75-100, based on the Minimum Completeness Criteria (KKM), supported by objective justifications.
    The entire process will adhere to academic standards.\nFinal Grade: [Score]
    """

    messages = [
        ChatMessage(
            role="system", content=prompt,
            temperature=0.8,
            max_tokens=24576,
            top_p=1,
        ),
        ChatMessage(role="user", content=request.input_text),
    ]

    resp = llm.chat(messages)

    # Extracting the Final Grade using regex
    content = resp.message.content
    final_grade_match = re.search(r'Final Grade:\s*(\d+)', content)
    if final_grade_match:
        final_grade = final_grade_match.group(1)

    # Removing the original "Final Grade" line from the content
    resp.message.content = re.sub(r'Final Grade:\s*\d+\n\n', '', content)


    # Assuming `resp` contains the response from the LLM in the format you expect
    # You might need to adjust the below line based on the actual structure of `resp`
    response = {"message": "Processed text", "result": resp.message.content, "final_grade":final_grade}
    return response

# Start the Server
def start_ngrok():
    url = ngrok.connect(8000, hostname="select-visually-ram.ngrok-free.app")
    print('ngrok tunnel "public_url" -> "http://127.0.0.1:8000"')
    print(f"Public URL: {url}")

# Set the environtment
if __name__ == "__main__":
    ngrok.set_auth_token("2hHxvO2eUfWGhtbxk8HLVA76943_6Jqg3i743gk3b93LFSBnQ")
    start_ngrok()
    uvicorn.run(app, port=8000)

INFO:     Started server process [212]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


ngrok tunnel "public_url" -> "http://127.0.0.1:8000"
Public URL: NgrokTunnel: "https://select-visually-ram.ngrok-free.app" -> "http://localhost:8000"
INFO:     103.67.78.86:0 - "POST /v1/grading-similarity-document HTTP/1.1" 200 OK
INFO:     103.67.78.86:0 - "POST /v1/grading-similarity-document HTTP/1.1" 200 OK
INFO:     103.67.78.86:0 - "POST /v1/grading-recommendation HTTP/1.1" 200 OK
INFO:     103.67.78.86:0 - "POST /v1/grading-recommendation HTTP/1.1" 200 OK
INFO:     103.67.78.86:0 - "POST /v1/grading-recommendation HTTP/1.1" 200 OK
INFO:     103.67.78.86:0 - "POST /v1/grading-recommendation HTTP/1.1" 200 OK


# Testing
Resource Independently Testing Features:

RAKE:
https://colab.research.google.com/drive/1utMUHccXoQZqnRo3r9oUdHj8D-SnaFV7#scrollTo=VB9mOqGvVxt_

Groq-chat:
https://colab.research.google.com/drive/1DZ0ZWDf4LqznrfeYKuNU1t9AdXFAxMLz?usp=sharing

Make-Quiz-Check-Answer
https://colab.research.google.com/drive/1v8Z8a0ov4C5S7MPo7RJ4tvMDlYFCIbPS?usp=sharing




In [ ]:
# This is testing of the
# def process_text(input_text):
#     llm = Groq(model="mixtral-8x7b-32768", api_key="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")

#     prompt = """
#     As a specialized assistant in teaching and evaluating narrative text structure, your task entails assessing the provided narrative based on Orientation, Complication, Resolution, and Reorientation,
#     devoid of extensive preamble. Each segment will be scrutinized for quality, with specific examples provided, and at least five detailed suggestions for improvement offered. Grammatical or syntactical
#     errors will be identified, along with clear recommendations for correction. The significance of each mistake will be highlighted, elucidating how rectifying them could enhance the overall text quality.
#     The evaluation will culminate in assigning a final grade, within even numbers ranging between 75-100, based on the Minimum Completeness Criteria (KKM), supported by objective justifications.
#     The entire process will adhere to academic standards.\nFinal Grade: [Score]
#     """

#     messages = [
#         ChatMessage(
#             role="system", content=prompt,
#             temperature=0.8,
#             max_tokens=24576,
#             top_p=1,
#         ),
#         ChatMessage(role="user", content=input_text),
#     ]

#     resp = llm.chat(messages)
#     # Assuming `resp` contains the response from the LLM in the format you expect
#     # You might need to adjust the below line based on the actual structure of `resp`
#     response = {"message": "Processed text", "result": resp}
#     return response

# input_text = """
# Once upon a time, in the vast and endless sky, there lived a joyful cloud named Nimbus. Nimbus enjoyed a unique friendship with both the Sun, who shone brightly day after day, and the Earth, a vibrant planet teeming with life. Every morning, Nimbus would gleefully catch the Sun's first rays, turning brilliant shades of orange, pink, and gold. As the day wore on, Nimbus would drift over the Earth's sprawling landscapes, from towering mountains and lush forests to arid deserts and bustling cities. The Earth cherished Nimbus, for he carried life-giving water, showering it upon the land when it was most needed. Together, the Cloud, the Sun, and the Earth lived in harmonious balance, each playing a vital role in the cycle of life.
# Once in the celestial realms, where the horizon meets infinity, floated a wise cloud named Nimbus. This cloud was not just any cloud; Nimbus had seen centuries unfold below, witnessing the dance of the seasons across the Earth's face. Each day, Nimbus greeted the Sun, whose eternal flame ignited life beneath. The Sun's rays, a golden tapestry, wove through Nimbus, casting shadows and light across the lands. As Nimbus journeyed across the skies, it whispered to the rivers, mountains, and forests below, nurturing them with rain, orchestrating the symphony of nature.
# There once was a cloud, Nimbus, who roamed the vast skies. Unlike any ordinary cloud, Nimbus harbored a secret: it could change shapes at will. This talent brought immense joy to the creatures of the Earth, who often looked up to find animals, trees, and even faces formed in the sky. The Sun, ever so generous, lent its splendid colors to Nimbus, painting the dawn and dusk with hues of reds and purples. Together, Nimbus and the Sun were the artists of the sky, creating masterpieces that spanned the horizon for all on Earth to see.
# Far above the bustling life on Earth, Nimbus, a playful cloud, frolicked in the boundless sky. Each morning, Nimbus raced the Sun's rays to the edge of the world, bathing in their amber glow. The Earth below, a patchwork of greens and blues, flourished under their careful watch. Nimbus loved to shape itself into fluffy animals, delighting the children who gazed up from below. The Sun, with its gentle warmth, and Nimbus, with its cooling showers, balanced the world in harmony, ensuring the cycle of life continued unabated.
# In the early hours before dawn, when the world whispered secrets, Nimbus, a curious cloud, hovered close to the Earth, listening intently. As the Sun peeked over the horizon, it filled Nimbus with a radiant glow, casting a soft light over the sleeping Earth. Nimbus, moved by the Earth's silent beauty, would then roam the skies, deciding where to bestow its life-giving bounty. The Sun's warmth coaxed the best out of Nimbus, allowing it to nourish the land below. Together, they worked in unison, a testament to the unwavering partnership between the sky and the ground.
# As Nimbus floated over ancient cities and untouched wilds, it encountered another cloud, Zephyr, born of a distant storm and carrying tales of far-off lands. Together, they wandered the skies, sharing stories under the Sun's watchful gaze. One day, Zephyr spoke of a desert so vast that it seemed to swallow the horizon, a place where rain had not touched the earth for a century. Moved by the story, Nimbus decided it was time to embark on a new journey. With the Sun blessing their path, Nimbus and Zephyr set off to find this desert, determined to bring life back to its barren sands.
# One evening, as Nimbus was painting the sunset, it noticed that the colors were not as bright as usual. The Sun seemed tired, its light dimming. Concerned, Nimbus approached the Sun, who shared that it felt its energy waning, a phenomenon that happened once every millennia for a short period. Understanding the importance of the Sun's vitality to the Earth, Nimbus pledged to help. It began gathering moisture more diligently than ever, creating a mirror-like surface on the Earth below to reflect the Sun's light, helping to conserve its energy. This act of solidarity reminded all of Earth's inhabitants of the interdependence among the elements.
# After many seasons of harmony, Nimbus grew restless. It longed to explore beyond the familiar landscapes and the Sun's reach. One twilight, a comet streaked across the sky, igniting Nimbus's imagination with possibilities of the cosmos. With a heavy heart, Nimbus informed the Sun and the Earth of its desire to venture into the unknown. Supported by their blessings, Nimbus ascended, propelled by cosmic winds, leaving a trail of mist in its wake. As it journeyed through the stars, Nimbus discovered celestial wonders but also learned that no beauty was quite like that of its home, the Earth."""
# print(process_text(input_text)["result"])